In [1]:
import nltk
import os
from os import getcwd
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples

In [2]:
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\Rayan_Pardaz\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [3]:
for name in twitter_samples.fileids():
    print(name)

negative_tweets.json
positive_tweets.json
tweets.20150430-223406.json


In [4]:
negatives = twitter_samples.strings('negative_tweets.json')
positives = twitter_samples.strings('positive_tweets.json')
all_tweets = twitter_samples.strings('tweets.20150430-223406.json')

convert data to dataframe

In [5]:

df_positives = pd.DataFrame(positives)
df_negatives = pd.DataFrame(negatives)
df_positives['label'] = 1
df_negatives['label'] = 0
df = pd.concat([df_positives , df_negatives] , ignore_index= True)
df = df.sample(frac = 1).reset_index(drop = True)

In [6]:
df.head(20)

,0,label
0,@silvslee lol go! It's cheap right now :),1
1,i miss 5sos so much i cant deal with waiting p...,0
2,:( I went to bed way too late,0
3,@treybaimbridge @hdez_emmy @IAmAlii1 @Abir_Khu...,1
4,@1Erre I'll take the infection :),1
5,"@Jaaeeeeee lucky :( I was feeling hungry, but ...",0
6,@MrsManfyDiston Thanks for following! Do get i...,1
7,What do we reckon are the chances of me seeing...,0
8,"i may look strong, hey :-(",0
9,@JavsNH sure :),1


preparing dataset

In [7]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [8]:
df[0].iloc[1]

'i miss 5sos so much i cant deal with waiting possibly a year before i see them again :('

deleting noises

In [9]:
df[0] = df[0].apply(lambda x: re.sub(r'\$\w*', '', x))
df[0] = df[0].apply(lambda x: re.sub(r'^RT[\s]+', '', x))
df[0] = df[0].apply(lambda x: re.sub(r'https?:\/\/.*[\r\n]*', '', x))
df[0] = df[0].apply(lambda x: re.sub(r'#', '', x))


tokenization

In [10]:
df['tokens']= df[0].str.split()
print(df['tokens'])

0       [@silvslee, lol, go!, It's, cheap, right, now,...
1       [i, miss, 5sos, so, much, i, cant, deal, with,...
2                  [:(, I, went, to, bed, way, too, late]
3       [@treybaimbridge, @hdez_emmy, @IAmAlii1, @Abir...
4                [@1Erre, I'll, take, the, infection, :)]
                              ...                        
9995                                    [Great, News, :)]
9996    [@theCIGirl, @LeRoiHaptalon, @stpierjane, @Hea...
9997    [@_ESCN0016, @Onatsss, @ljabasula, @coracarona...
9998       [@clivewalker, near, enough, to, Dartmoor, :)]
9999    [@Hayleychinney, Hope, this, isn't, a, regular...
Name: tokens, Length: 10000, dtype: object


deleting stop_words

In [11]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])
print(df['tokens'])

0           [@silvslee, lol, go!, It's, cheap, right, :)]
1       [miss, 5sos, much, cant, deal, waiting, possib...
2                           [:(, I, went, bed, way, late]
3       [@treybaimbridge, @hdez_emmy, @IAmAlii1, @Abir...
4                     [@1Erre, I'll, take, infection, :)]
                              ...                        
9995                                    [Great, News, :)]
9996    [@theCIGirl, @LeRoiHaptalon, @stpierjane, @Hea...
9997    [@_ESCN0016, @Onatsss, @ljabasula, @coracarona...
9998           [@clivewalker, near, enough, Dartmoor, :)]
9999    [@Hayleychinney, Hope, regular, problem, :(, C...
Name: tokens, Length: 10000, dtype: object


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rayan_Pardaz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


stemming

In [12]:
nltk.download('wordnet')
stemmer = PorterStemmer()
df['stemmed'] = df['tokens'].apply(lambda x: [stemmer.stem(word) for word in x])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Rayan_Pardaz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
print(df['tokens'] , df['stemmed'])

0           [@silvslee, lol, go!, It's, cheap, right, :)]
1       [miss, 5sos, much, cant, deal, waiting, possib...
2                           [:(, I, went, bed, way, late]
3       [@treybaimbridge, @hdez_emmy, @IAmAlii1, @Abir...
4                     [@1Erre, I'll, take, infection, :)]
                              ...                        
9995                                    [Great, News, :)]
9996    [@theCIGirl, @LeRoiHaptalon, @stpierjane, @Hea...
9997    [@_ESCN0016, @Onatsss, @ljabasula, @coracarona...
9998           [@clivewalker, near, enough, Dartmoor, :)]
9999    [@Hayleychinney, Hope, regular, problem, :(, C...
Name: tokens, Length: 10000, dtype: object 0             [@silvsle, lol, go!, it', cheap, right, :)]
1       [miss, 5so, much, cant, deal, wait, possibl, y...
2                           [:(, i, went, bed, way, late]
3       [@treybaimbridg, @hdez_emmi, @iamalii1, @abir_...
4                         [@1err, i'll, take, infect, :)]
                             

Vectorization

In [14]:
tfidf = TfidfVectorizer(ngram_range=(1, 2), max_features=7000, stop_words='english', min_df=2)
df['stemmed'] = df['stemmed'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
x_tfidf = tfidf.fit_transform(df['stemmed'])
x_tfidf = x_tfidf.toarray()
label = df['label']

trainx, valx, trainy, valy = train_test_split(x_tfidf, label, test_size=0.2, random_state=42)

trainy = trainy.values.reshape((8000,1))
valy = valy.values.reshape((2000,1))


Normalize Features to bring all TF-IDF values to a similar range

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
trainx = scaler.fit_transform(trainx)
valx = scaler.transform(valx)

In [16]:
valx.shape

(2000, 6641)

In [17]:
valy.shape

(2000, 1)

In [18]:
def initialization(dim):
    w = np.random.randn(dim, 1) * 0.01
    b = 0

    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w,b

In [19]:
def sigmoid(z):
    s = 1/(1+np.exp(-z))
    return s

In [20]:
def propagate(w, x, b, y):
    m = x.shape[1]
    A = sigmoid(np.dot(w.T,x) + b) 
    cost = np.sum(((- np.log(A))*y + (-np.log(1-A))*(1-y)))/m  

    dw = (np.dot(x,(A-y).T))/m
    db = (np.sum(A-y))/m

    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())

    
    backs = {"dW" : dw,
             "dB" : db}
    return cost, backs

In [21]:
w, b, X, Y = np.array([[1.],[2.]]), 2., np.array([[1.,2.,-1.],[3.,4.,-3.2]]), np.array([[1,0,1]])
cost, backs = propagate(w, X, b, Y)
print ("dw = " + str(backs["dW"]))
print ("db = " + str(backs["dB"]))
print ("cost = " + str(cost))


dw = [[0.99845601]
 [2.39507239]]
db = 0.001455578136784208
cost = 5.801545319394553


In [22]:
def optimize(w, b, x, y, learning_rate, num_iteration):

    costs = []
    for i in range(num_iteration):
        cost, backs = propagate(w, x, b, y)
        dw = backs["dW"]
        db = backs["dB"]
        w = w - (learning_rate*dw)
        b = b - (learning_rate*db)  

        if i%100 == 0:
            costs.append(cost)

        params = {"w": w,
                  "b": b} 
        grads = {"dw": dw,
                 "db": db}   
    return costs, params, grads

In [23]:
costs, params, grads = optimize(w, b, X, Y, learning_rate = 0.009, num_iteration= 100)

print ("w = " + str(params["w"]))
print ("b = " + str(params["b"]))
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))

w = [[0.19033591]
 [0.12259159]]
b = 1.9253598300845747
dw = [[0.67752042]
 [1.41625495]]
db = 0.21919450454067654


In [24]:
def predict(w, x, b):

    m = x.shape[1]
    y_prediction = np.zeros((1,m))
    w = w.reshape(x.shape[0], 1)
    A = sigmoid(np.dot(w.T , x) + b)
    y_prediction = (A >= 0.5) * 1.0
    assert(y_prediction.shape == (1,m))

    return y_prediction

In [25]:
w = np.array([[0.1124579],[0.23106775]])
b = -0.3
X = np.array([[1.,-1.1,-3.2],[1.2,2.,0.1]])
print ("predictions = " + str(predict(w, X, b)))

predictions = [[1. 1. 0.]]


In [26]:
def model(X_train, y_train, X_test, y_test, learning_rate, num_iteration):
    
    w, b = initialization((X_train.shape[0]))
    costs, params, grads = optimize(w, b, X_train, y_train, learning_rate, num_iteration)

    w = params["w"]
    b = params["b"]

    y_prediction_train = predict(w, X_train, b)
    y_prediction_test = predict(w, X_test, b)

    print(f"train accuracy : {100 - np.mean(np.abs(y_train - y_prediction_train)) * 100:.6f}")
    print(f"test accuracy : {100 - np.mean(np.abs(y_test - y_prediction_test)) * 100:.6f}")

    d = {"costs" : costs,
         "y_prediction_test": y_prediction_test, 
         "y_prediction_train" : y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iteration": num_iteration}
    
    return d

In [27]:
trainx_transpose = trainx.T
trainy_transpose = trainy.T
valx_transpose = valx.T
valy_transpose = valy.T

In [28]:
d = model(trainx_transpose, trainy_transpose, valx_transpose, valy_transpose, learning_rate = 0.01,  num_iteration = 5000)

train accuracy : 94.837500
test accuracy : 69.650000
